In [1]:
from qSUN.qCircuit import *
# from qSUN.qGATES import *
import time
import numpy as np
import cmath
import math

In [2]:
def isKthBitSet(i, k, n):
    if i & (1 << (n-k-1)) == 0:
        return True
    else:
        return False

In [3]:
def Xsquare(wavefunction, n):
    """a square root of the NOT gate."""
    states = wavefunction.state
    amplitude = wavefunction.amplitude
    qubit_num = len(states[0])
    new_amplitude = np.zeros(2**qubit_num, dtype = complex)
    cut = 2**(qubit_num-n-1)
    if n >= qubit_num or n < 0:
        raise TypeError("Index is out of range")
    for i in np.nonzero(amplitude)[0]:
        new_amplitude[i] += (1+1j)*amplitude[i]/2
        if states[i][n] == '0':
            new_amplitude[i+cut] += (1-1j)*amplitude[i]/2
        else:
            new_amplitude[i-cut] += (1-1j)*amplitude[i]/2  
    wavefunction.amplitude = new_amplitude
    (wavefunction.visual).append([n, 'XS'])

In [4]:
def CNOT(wavefunction, control, target):
    """Flip target if control is |1>: 
    math:`P_0 \otimes I + P_1 \otimes X = \begin{pmatrix} 1&0&0&0 \\ 0&1&0&0 \\
                                            0&0&0&1 \\ 0&0&1&0 \end{pmatrix}`"""
    states = wavefunction.state
    amplitude = wavefunction.amplitude
    qubit_num = len(states[0])
    new_amplitude = np.zeros(2**qubit_num, dtype = complex)
    if control < target or control > target:
        cut = 2**(qubit_num-target-1)
    else:
        raise TypeError("Control qubit and target qubit must be distinct")
    for i in np.nonzero(amplitude)[0]:
        if states[i][control] == '1':
            if states[i][target] == '0':
                new_amplitude[i+cut] += amplitude[i]
            else:
                new_amplitude[i-cut] += amplitude[i]
        else:
            new_amplitude[i] = amplitude[i]
    wavefunction.amplitude = new_amplitude
    (wavefunction.visual).append([control, target, 'CX'])

In [5]:
def H(wavefunction, n):
    """Hadamard gate: math:`\frac{1}{\sqrt{2}} \begin{pmatrix} 1 & 1 \\ 1 & -1 \end{pmatrix}`"""
    states = wavefunction.state
    amplitude = wavefunction.amplitude
    qubit_num = len(states[0])
    new_amplitude = np.zeros(2**qubit_num, dtype = complex)
    cut = 2**(qubit_num-n-1)
    if n >= qubit_num or n < 0:
        raise TypeError("Index is out of range")
    for i in range(0, 2**qubit_num):
        if states[i][n] == '0':
            new_amplitude[i] += amplitude[i]/2**0.5
            new_amplitude[i+cut] += amplitude[i]/2**0.5
        else:
            new_amplitude[i] -= amplitude[i]/2**0.5
            new_amplitude[i-cut] += amplitude[i]/2**0.5  
    wavefunction.amplitude = new_amplitude

In [6]:
def H_new(wavefunction, n):
    """Hadamard gate: math:`\frac{1}{\sqrt{2}} \begin{pmatrix} 1 & 1 \\ 1 & -1 \end{pmatrix}`"""
    states = wavefunction.state
    amplitude = wavefunction.amplitude
    qubit_num = len(states[0])
    new_amplitude = np.zeros(2**qubit_num, dtype = complex)
    n = (qubit_num-n-1)
    if n >= qubit_num or n < 0:
        raise TypeError("Index is out of range")
    for i in range(0, 2**(qubit_num-1)):
        s = math.floor(i/2**n)*2**(n+1)+(i%(2**n))
        new_amplitude[s] += amplitude[s]/2**0.5 + amplitude[s+2**n]/2**0.5
        new_amplitude[s+2**n] += amplitude[s]/2**0.5 - amplitude[s+2**n]/2**0.5
    wavefunction.amplitude = new_amplitude

In [7]:
def qvm_circuit(n_qubit, depth):
    circuit = Qubit(n_qubit)
    for m in range(depth):
        for i in range(n_qubit):
            H(circuit, i)
#             Xsquare(circuit, i)
#         for i in range(1, n_qubit):
#             CNOT(circuit, i, 0)
    return circuit.probabilities()

In [8]:
def qvm_circuit_new(n_qubit, depth):
    circuit = Qubit(n_qubit)
    for m in range(depth):
        for i in range(n_qubit):
            H_new(circuit, i)
#             Xsquare(circuit, i)
#         for i in range(1, n_qubit):
#             CNOT(circuit, i, 0)
    return circuit.probabilities()

In [9]:
qubit_min = 2
qubit_max = 10
depth = 11

In [10]:
data_qvm = []
for n_qubit in range(qubit_min, qubit_max):
    start = time.time()
    qvm_circuit(n_qubit, depth)
    data_qvm.append(time.time()-start)

In [11]:
data_qvm_new = []

for n_qubit in range(qubit_min, qubit_max):
    start = time.time()
    qvm_circuit_new(n_qubit, depth)
    data_qvm_new.append(time.time()-start)

In [17]:
import pandas as pd

df = pd.DataFrame({'H': data_qvm, 'New H': data_qvm_new}, index = range(qubit_min, qubit_max))
df

,H,New H
2,0.000000,0.000000
3,0.000997,0.000997
4,0.000997,0.000997
5,0.002992,0.003989
6,0.007978,0.008978
7,0.017954,0.018948
8,0.038895,0.044881
9,0.085770,0.101727


In [17]:
N = 3
n = 1
n = (N-n-1)

for i in range(0, 2**(N-1)):
    s = math.floor(i/2**n)*2**(n+1)+(i%(2**n))
    print(s, s + 2**n)

0 2
1 3
4 6
5 7


In [6]:
n = 0
for i in range(0, 2**N):
    if isKthBitSet(i, n, N):
        print(i, i+2**(N-n-1))

0 4
1 5
2 6
3 7


In [45]:
n = 3
circuit = Qubit(n)

H(circuit, 0)
H(circuit, 1)
H(circuit, 2)

H_new(circuit, 1)
H_new(circuit, 2)
H_new(circuit, 0)

circuit.print_state()

'(0.9999999999999996+0j)|000> + 0j|001> + 0j|010> + 0j|011> + 0j|100> + 0j|101> + 0j|110> + 0j|111>'

In [46]:
circuit.probabilities()

array([1., 0., 0., 0., 0., 0., 0., 0.])

In [47]:
n = 3
circuit = Qubit(n)

H(circuit, 0)
H(circuit, 1)
H(circuit, 2)

H(circuit, 1)
H(circuit, 2)
H(circuit, 0)

circuit.print_state()

'(0.9999999999999996+0j)|000> + 0j|001> + 0j|010> + 0j|011> + 0j|100> + 0j|101> + 0j|110> + 0j|111>'